In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import scipy
from scipy.stats import chisquare
import sys
import copy

In [2]:
exchange_rate_USD_EUR = 0.9

exchange_rate_GBP_EUR = 1.19

currency = ["EUR", "GBP", "USD"]


In [3]:
def generate_data(n):

    positions = np.zeros((n,3))

    for i in range(n):
        positions[i,0] = random.randint(-1000,1000)
        positions[i,1] = random.randint(1,3) # 1 = IRD, 2 = EQD, 3 = Curr.
        
        if positions[i,1] == 3:
            positions[i,2] = random.randint(1,3) # 1 = EUR, 2 = GBP, 3 = USD
    
    
    '''
    positions = np.zeros((2,3))

    positions[0,0] = 1000
    positions[1,0] = -500

    positions[0,1] = 3
    positions[1,1] = 3

    positions[0,2] = 1
    positions[1,2] = 2
    '''

    print(positions)
    return positions


In [4]:
def settlementrisk(position):

    risk = 0
    mod = position[0].copy()
    if position[1] == 3:

        if position[2] == 1:
            mod *= exchange_rate_GBP_EUR
        elif position[2] == 2:
            mod *= exchange_rate_USD_EUR
    
        return max(0,float(mod))

    return max(0, float(mod))             
    


In [5]:
def BruteForce(positions):

    netting_GBP = np.zeros((0, 3))
    netting_USD = np.zeros((0, 3))
    netting_EUR = np.zeros((0, 3))
    remaining_impacts = np.zeros((0, 3))

    for i in range(len(positions)):

        if positions[i,2] == 3:
            row = copy.deepcopy(positions[i,:])
            netting_EUR = np.vstack([netting_EUR, row])
        elif positions[i,2] == 1:
            row = copy.deepcopy(positions[i,:])
            netting_GBP = np.vstack([netting_GBP, row])
        elif positions[i,2] == 2:
            row = copy.deepcopy(positions[i,:])
            netting_USD = np.vstack([netting_USD, row])
        else:
            row = copy.deepcopy(positions[i,:])
            remaining_impacts = np.vstack([remaining_impacts, row])

    
    return settlementrisk(np.array([np.sum(netting_GBP[:,0]), 3, 1])) + settlementrisk(np.array([np.sum(netting_USD[:,0]), 3, 2])) + settlementrisk(np.array([np.sum(netting_EUR[:,0]), 3, 3])) + settlementrisk(np.array([np.sum(remaining_impacts[:,0]),0,0]))


In [6]:
def Linearisation(positions):

    baseline = copy.deepcopy(positions[0,:])
    baseline_exposure = settlementrisk(baseline)
    total = baseline_exposure
    
    for i in range(len(positions)-1):

        row = copy.deepcopy(positions[i+1,:])

        if baseline[1] == row[1] and baseline[1] == 3.:

            if baseline[2] == row[2]:
                
                temp_netting = np.array([0,0,0])
                temp_netting = np.vstack([temp_netting, baseline])
                temp_netting = np.vstack([temp_netting, row])

                sum = np.sum(temp_netting[:,0])
                calc = settlementrisk(np.array([sum, 3, row[2]]))
                total += calc - baseline_exposure
            else:
                total += settlementrisk(row)

        else:
            total += settlementrisk(row)

    return total        

In [7]:
def Conservative(positions):

    total = 0
    for i in range(len(positions)):
        row = copy.deepcopy(positions[i,:])
        total += settlementrisk(row)
    
    return total


In [8]:
def Averages(positions):

    n = len(positions)-1
    baseline = copy.deepcopy(positions[0,:])
    baseline_exposure = settlementrisk(baseline)
    total = baseline_exposure
    
    for i in range(len(positions)-1):

        row = copy.deepcopy(positions[i+1,:])

        if baseline[1] == row[1] and baseline[1] == 3.:
            
            if baseline[2] == row[2]:

                temp_netting = np.array([0,0,0])
                temp_netting = np.vstack([temp_netting, baseline])
                temp_netting = np.vstack([temp_netting, row])
                temp_netting[2,0] *= n

                total += 1/n*(settlementrisk(np.array([np.sum(temp_netting[:,0]), 3, row[2]])) - baseline_exposure)
            else:
                row[0] *= n
                total += 1/n*settlementrisk(row)

        else:
            row[0] *= n
            total += 1/n*settlementrisk(row)

    return total

In [9]:
def sim(n):

    data = generate_data(n)

    total_BF = BruteForce(data)

    total_Linearisation = Linearisation(data)

    total_Conservative = Conservative(data)

    total_Averages = Averages(data)

    print("Brute Force: ", total_BF)
    print("\n")
    print("Linearisation: ", total_Linearisation)
    print("\n")
    print("Conservative: ", total_Conservative)
    print("\n")
    print("Averages: ", total_Averages)

In [10]:
sim(20)

[[ 159.    3.    1.]
 [ 499.    3.    2.]
 [ 837.    1.    0.]
 [-930.    3.    1.]
 [ 915.    2.    0.]
 [-350.    3.    3.]
 [ 493.    2.    0.]
 [ 535.    1.    0.]
 [-100.    1.    0.]
 [ 101.    1.    0.]
 [-569.    2.    0.]
 [-330.    2.    0.]
 [-842.    3.    3.]
 [ 980.    3.    2.]
 [-754.    1.    0.]
 [ 858.    2.    0.]
 [-375.    2.    0.]
 [ 645.    1.    0.]
 [ 305.    1.    0.]
 [-573.    2.    0.]]
Brute Force:  3319.1000000000004


Linearisation:  6020.1


Conservative:  6209.3099999999995


Averages:  6199.351578947369
